# TextRank로 크롤링한 뉴스 기사 요약 모델 만들기

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from urllib.request import urlretrieve
import re
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd

In [21]:
import psycopg2
from datetime import datetime

In [22]:
db_connection = psycopg2.connect(
    host="localhost", #ip
    dbname="final_project",
    user="postgres",
    password="sesac0916",
    port=5432
)

cursor = db_connection.cursor()
# SELECT 쿼리 실행
cursor.execute("SELECT * FROM final_project.politics;")
# 결과 가져오기
result = cursor.fetchall()
# 결과 출력 (예시로 처음 5개만 출력)
# 연결 종료
cursor.close()
db_connection.close()

In [27]:
df=pd.DataFrame(result, columns=['id', 'create_dt', 'news_name', 'news_genre', 'news_image_path', 'news_title', 'news_origin', 'news_url', 'news_summary'])

In [25]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [46]:
kkma = Kkma()

# text를 입력받아 kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
def text2sentences(text):
    sentences = kkma.sentences(text)
    for idx in range(0, len(sentences)):
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' ' + senteces[idx])
            sentences[idx] = ''
    return sentences

In [40]:
news_content = df['news_origin']

In [44]:
news_content

0      비주류, 당 지도부 희생 압박…금융 전문가 출신 洪 "객관적 주장도 당리당략 이유로...
1      39세 첫 당선 후 이번이 7선 도전\n\n이인제 전 의원이 지난 1월 6일 오후 ...
2      빌렘 국왕 초청 국빈만찬에 히딩크 전 감독 참석\n尹 "박지성 유럽 커리어 시작과 ...
3      비윤계도 거취 의견 수렴…14일 최고위서 거취 표명\n‘사퇴·험지 출마’ ‘대표 유...
4      이준석 전 국민의힘 대표 페이스북 글\n이준석 전 국민의힘 대표가 11월 30일 국...
                             ...                        
193    국민의힘 내부에선 12일 총선 불출마 결단을 한 장제원 의원과 같은 친윤(親윤석열)...
194    CBS노컷뉴스 취재진이 입수한, 정의기억연대에 통일부가 보낸 사전접촉신고 수리 거부...
195    4·10 총선 레이스 시작\n국회서 공식 불출마 선언…중진들 ‘거부’ 명분 사라져\...
196    선거구 모르는 채 예비후보 등록‥정치 신인들은 '난감'\nMBC뉴스\n재생\n7,4...
197    장제원 국민의힘 의원이 12일 국회 소통관에서 22대 총선 불출마 선언 기자회견을 ...
Name: news_origin, Length: 198, dtype: object

In [47]:
sentences = text2sentences(news_content[0])

In [51]:
twitter = Twitter()
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  ]))
    return nouns

<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\user\anaconda3\Lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\AppData\Local\Temp\ipykernel_15664\2706461304.py:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [52]:
nouns = get_nouns(sentences)

In [53]:
nouns

['비주류 당 도부 희생 압박 금융 전문가 출신 객관 주장 리 당략 이유 더 불어 민주당 홍성 국 의원 내년 총선 불출 선언',
 '미래 셋 대우 현 미래 셋 증권 사장 출신 총선 세종 시 갑 전략 공천 홍 의원 불출 결심 이유 저 지난 간 국회의원 나름 시각 우리 사회 노력 지금 후진 적 정치 구조 가지 한계 성과 내지 고 말 고 토로',
 '홍 의원 불 출마 선언문 통해 저 지난 간 국회의원 나름 시각 우리 사회 노력 며 전환 경고 대안 것 제 정치 목적 이자 소임 생각 고 말',
 '그 한국 사회 양극화 해소 저 출생 고령화 기후변화 국토 균형 발전 산업 구조 전환 국민 연금 건강 보험 개혁 등 혁명 수준 문제 산적 며 때로는 객관 주장 도 리 당략 이유 고',
 '제로 섬 정치 오히려 사회 불평등 심화 민주주의 위기 며 윤석 열 정부 이념 우리나라 후진 고 지적',
 '한계 대해 시간 고민 저 국회의원 국민 직접 소통 우리나라 미래 미래 학 연구자 로 다시 고 말',
 '홍 의원 제 수 분야 역할 것 국가 위 더 나은 길 라며 민주 당원 정책 우리 당 사회 제안 인 싱크 탱크 역할 고',
 '그 제 수 분야 역할 것 국가 위 더 나은 길이 생각 며 세종 시민 당원 동지 여러분 그동안 저 사랑 진심',
 '사명 가지 못 데 대해 사죄 말씀 고',
 '총선',
 '전날 국민 힘 중진 장 제원 의원 날 홍 의원 불출 선언 당 혁신 경쟁 본격 전개 것',
 '다만 국민 힘 선 중 진급 불 출마 험지 출마 논의 것 달리 민주당 선 초선 의원 몇몇 불출 고심 중인 것 혁신 경쟁 지적',
 '현재 민주당 불 출마 선언 의원 반기 국회의장 박 병석 의원 선 제외 상호 의원 선 오 영 환 강 민정 초 선 의원 홍 의원 명',
 '민주당 비주류 혁신 실패 총선 필 패 를 명분 당 도부 희생 요구 목소리',
 '당내 비주류 의원 모임 원칙 상식 소속 응 천 의원 전날 라디오 인터뷰 국민 힘 인 요한 혁신 위 보고서 대해 앞 질서 반영 고 진압된 것 보이 찻잔 속 태풍 것 

In [55]:
nouns = get_nouns(sentences)
print(sentences[0])
print()
print(nouns[0])

비주류, 당 지도부 희생 압박… 금융 전문가 출신 洪 " 객관적 주장도 당리 당략 이유로 폄하돼" 더 불어 민주당 홍성 국 의원이 내년 총선 불출마를 선언했다.

비주류 당 도부 희생 압박 금융 전문가 출신 객관 주장 리 당략 이유 더 불어 민주당 홍성 국 의원 내년 총선 불출 선언


In [56]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [57]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence
sent_graph = build_sent_graph(nouns)

In [58]:
print(sent_graph)

[[1.         0.14424148 0.0161064  0.16026822 0.         0.
  0.01688703 0.         0.         0.19458605 0.14729162 0.07874228
  0.15263239 0.30479111 0.07447675 0.         0.08491931 0.29172376]
 [0.14424148 1.         0.29016161 0.07447216 0.0989913  0.25583673
  0.07122544 0.04415216 0.06057035 0.14465731 0.06679431 0.03878753
  0.0603172  0.03838065 0.02895722 0.05163555 0.04265292 0.04153808]
 [0.0161064  0.29016161 1.         0.06416176 0.07161603 0.04642615
  0.09290332 0.04653518 0.         0.         0.04071619 0.0998436
  0.12172707 0.         0.03777053 0.06735113 0.05563459 0.05418044]
 [0.16026822 0.07447216 0.06416176 1.         0.02539182 0.02537033
  0.02448315 0.         0.         0.         0.03043872 0.02306549
  0.         0.         0.01882439 0.         0.         0.        ]
 [0.         0.0989913  0.07161603 0.02539182 1.         0.06247563
  0.03449798 0.         0.         0.         0.         0.05679986
  0.         0.         0.         0.07323626 0.     

In [59]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

words_graph, idx2word = build_words_graph(nouns)

In [60]:
print(words_graph)
print()
print(idx2word)

[[1.         0.         0.         ... 0.         0.40824829 0.        ]
 [0.         1.         0.         ... 0.70710678 0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.70710678 0.         ... 1.         0.         0.5       ]
 [0.40824829 0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.5        0.         1.        ]]

{74: '비주류', 40: '도부', 169: '희생', 102: '압박', 22: '금융', 126: '전문가', 153: '출신', 4: '객관', 136: '주장', 35: '당략', 119: '이유', 71: '불어', 58: '민주당', 167: '홍성', 117: '의원', 27: '내년', 150: '총선', 72: '불출', 86: '선언', 55: '미래', 38: '대우', 139: '증권', 78: '사장', 89: '세종', 125: '전략', 12: '공천', 6: '결심', 141: '지난', 17: '국회의원', 25: '나름', 95: '시각', 112: '우리', 80: '사회', 29: '노력', 140: '지금', 168: '후진', 130: '정치', 13: '구조', 0: '가지', 160: '한계', 88: '성과', 28: '내지', 156: '토로', 151: '출마', 87: '선언문', 157: '통해', 127: '전환', 7: '경고', 37: '대안', 52: '목적', 120: '이자', 91: '소임', 85: '생각', 

# TextRank 구현하기

In [61]:
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) #연립방적식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [62]:
sent_rank_idx = get_ranks(sent_graph) # sent_graph : sentence 가중치 그래프
sent_rank_idx

{0: 1.579591532154613,
 1: 1.6029306007631807,
 2: 1.1492272382786937,
 3: 0.5370085686315698,
 4: 0.5472577442493085,
 5: 0.7318638207335914,
 6: 0.9952284926336418,
 7: 0.597551793149984,
 8: 0.3937502468161183,
 9: 0.6595152927757132,
 10: 1.2881336048932919,
 11: 1.1383741642856502,
 12: 1.471007137919255,
 13: 1.174048408994206,
 14: 1.2404403677846052,
 15: 0.5230434693529995,
 16: 1.0837968247126766,
 17: 1.2872306918708925}

In [64]:
sorted_sent_rank_idx = sorted(sent_rank_idx, key = lambda k: sent_rank_idx[k], reverse=True)
print(sorted_sent_rank_idx)

[1, 0, 12, 10, 17, 14, 13, 2, 11, 16, 6, 5, 9, 7, 4, 3, 15, 8]


In [65]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

# 요약하기 실행

In [66]:
def summarize(sent_num=5):
    summary = []
    index= []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
    index.sort()
    
    for idx in index:
        summary.append(sentences[idx])
        
    for text in summary :
        print(text)
        print("\n")

In [80]:
summarize(4)

비주류, 당 지도부 희생 압박… 금융 전문가 출신 洪 " 객관적 주장도 당리 당략 이유로 폄하돼" 더 불어 민주당 홍성 국 의원이 내년 총선 불출마를 선언했다.


미래에 셋 대우( 현 미래에 셋증권) 사장 출신으로 21대 총선에서 세종 시 갑에 전략 공천된 홍 의원은 불출마를 결심한 이유로 " 저는 지난 4년 간 국회의원으로서 나름 새로운 시각으로 우리 사회를 바꿔 보려 노력했지만 지금의 후진 적인 정치 구조가 가지고 있는 한계로 인해 성과를 내지 못했다" 고 말했다" 고 토로 했다.


전날 국민의 힘 중진 장 제원 의원에 이어 이날엔 홍 의원이 불출마를 선언하면서 양당의 혁신 경쟁이 본격적으로 전개될 것으로 보인다.


현재까지 민주당에서 불출마 선언을 한 의원은 21대 전반기 국회의장을 지낸 박 병석 의원 (6 선) 을 제외하면 우 상호 의원 (4 선), 오 영 환· 강 민정( 초 선) 의원, 홍 의원까지 4명이다.




In [71]:
def keywords(word_num=10):
    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)
        
    for idx in index:
        keywords.append(idx2word[idx])
        
    print(keywords)

In [73]:
keywords()

['의원', '사회', '국민', '이유', '불출', '구조', '민주당', '정치', '우리', '혁신']


In [82]:
sentences

['비주류, 당 지도부 희생 압박… 금융 전문가 출신 洪 " 객관적 주장도 당리 당략 이유로 폄하돼" 더 불어 민주당 홍성 국 의원이 내년 총선 불출마를 선언했다.',
 '미래에 셋 대우( 현 미래에 셋증권) 사장 출신으로 21대 총선에서 세종 시 갑에 전략 공천된 홍 의원은 불출마를 결심한 이유로 " 저는 지난 4년 간 국회의원으로서 나름 새로운 시각으로 우리 사회를 바꿔 보려 노력했지만 지금의 후진 적인 정치 구조가 가지고 있는 한계로 인해 성과를 내지 못했다" 고 말했다" 고 토로 했다.',
 '홍 의원은 13일 불출마 선언문을 통해 " 저는 지난 4년 간 국회의원으로서 나름 새로운 시각으로 우리 사회를 바꿔 보려 노력했다 "며 " 대전환을 경고하고 대안을 만드는 것이 제가 정치를 하는 목적 이자 소임이라고 생각했다" 고 말했다.',
 '그는 " 한국 사회는 양극화 해소, 저 출생· 고령화, 기후변화, 국토 균형 발전, 산업 구조 전환, 국민 연금· 건강 보험 개혁 등과 같은 혁명 수준으로 바꿔야 할 문제들이 산적해 있다 "며 " 때로는 객관적인 주장마저 도 당리 당략을 이유로 폄하 받기도 했다" 고 했다.',
 '이어 " 제로 섬 정치는 오히려 사회 불평등을 심화시키고, 민주주의를 위기에 빠뜨리고 있다 "며 " 특히 윤석 열 정부는 80년대 낡은 이념으로 우리나라를 후진시키고 있다" 고 지적했다.',
 '그러면서 " 이런 한계에 대해 오랜 시간 고민한 저는 국회의원보다는 국민과 직접 소통하고 우리나라의 미래 비전을 만드는 \' 미래 학 연구자\' 로 다시 돌아가려고 한다" 고 말했다.',
 '홍 의원은 " 제가 잘할 수 있는 분야에서 역할을 하는 것이 국가를 위하는 더 나은 길" 이라며 " 민주 당원으로서 좋은 정책을 만들어 우리 당과 사회에 제안하는 1 인 싱크 탱크 역할을 하려고 한다" 고 했다.',
 '그는 " 제가 잘할 수 있는 분야에서 역할을 하는 것이 국가를 위하는 더 나은 길이라 생각한다 "며 세종시민 그리고 당원 동지 여러분, 그동안 부족